In [1]:
import datetime
import os
import pandas as pd
from typing import Sequence

In [2]:
def read_partition(path: str) -> pd.DataFrame:
    def _read_partition(path: str, partition_cols: Sequence[Sequence[str]]) -> pd.DataFrame:
        if path.lower().endswith(".xlsx"):
            df = pd.read_excel(path)
            return [ df.assign(**dict(partition_cols)) ]
        elif os.path.isdir(path):
            return [
                df
                for subpath in os.listdir(path)
                for df in _read_partition(f"{path}/{subpath}", partition_cols + ([subpath.split("=")] if "=" in subpath else []))
            ]
        else:
            return []
    
    dfs = _read_partition(path, [])
    return pd.concat(dfs) if dfs else pd.DataFrame()


def parse_tamtop_datetime(input: str) -> datetime.datetime:
    return datetime.datetime.fromisoformat(
        input.replace(" ", "T").replace("(", "").replace(")", "")
    )

In [3]:
df = read_partition("data")
df["Time"] = df["Time"].apply(parse_tamtop_datetime)
df = df.set_index(["Device", "Time"]).drop(["NO."], axis=1)
display(df)

PM2.5(ug/m³)  AQI  CO₂(ppm)  TVOC  \
Device Time                                                           
03     2024-08-17 20:00:00+08:00          47.8  131       580    28   
       2024-08-17 21:00:00+08:00          61.1  155       493    67   
       2024-08-17 22:00:00+08:00          35.8  102       514    29   
       2024-08-17 23:00:00+08:00          35.5  101       513    18   
       2024-08-18 00:00:00+08:00          19.3   70       504    22   
...                                        ...  ...       ...   ...   
       2024-08-20 08:00:00+08:00           3.8   21       674    40   
       2024-08-20 09:00:00+08:00           4.1   23       667    41   
       2024-08-20 10:00:00+08:00          10.9   54       597    54   
       2024-08-20 11:00:00+08:00           9.1   51       610    57   
       2024-08-20 12:00:00+08:00           5.9   33       647    44   

                                  Temperature(℉)  Humidity(%RH) Floor  \
Device Time                                                             
03     2024-08-17 20:00:00+08:00            77.7             57     2   
       2024-08-17 21:00:00+08:00            81.9             51     2   
       2024-08-17 22:00:00+08:00            81.7             51     2   
       2024-08-17 23:00:00+08:00            80.1             53     2   
       2024-08-18 00:00:00+08:00            79.7             53     2   
...                                          ...            ...   ...   
       2024-08-20 08:00:00+08:00            78.8             60     2   
       2024-08-20 09:00:00+08:00            78.8             61     2   
       2024-08-20 10:00:00+08:00            79.0             61     2   
       2024-08-20 11:00:00+08:00            76.5             55     2   
       2024-08-20 12:00:00+08:00            75.6             53     2   

                                      Location  
Device Time                                     
03     2024-08-17 20:00:00+08:00  West Hallway  
       2024-08-17 21:00:00+08:00  West Hallway  
       2024-08-17 22:00:00+08:00  West Hallway  
       2024-08-17 23:00:00+08:00  West Hallway  
       2024-08-18 00:00:00+08:00  West Hallway  
...                                        ...  
       2024-08-20 08:00:00+08:00      Unit 210  
       2024-08-20 09:00:00+08:00      Unit 210  
       2024-08-20 10:00:00+08:00      Unit 210  
       2024-08-20 11:00:00+08:00      Unit 210  
       2024-08-20 12:00:00+08:00      Unit 210  

[130 rows x 8 columns]